In [2]:
!pip install numpy protobuf==3.16.0
!pip install onnx
!pip install -q simpletransformers
!pip install -q datasets transformers[sentencepiece] simpletransformers
!pip install onnxruntime

     |████████████████████████████████| 1.0 MB 14.3 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.


     |████████████████████████████████| 12.8 MB 15.2 MB/s 
     |████████████████████████████████| 248 kB 14.3 MB/s 
     |████████████████████████████████| 1.7 MB 82.8 MB/s 
     |████████████████████████████████| 311 kB 69.6 MB/s 
     |████████████████████████████████| 9.7 MB 63.3 MB/s 
     |████████████████████████████████| 1.2 MB 63.8 MB/s 
     |████████████████████████████████| 43 kB 2.2 MB/s 
     |████████████████████████████████| 3.5 MB 77.6 MB/s 
     |████████████████████████████████| 6.8 MB 80.0 MB/s 
     |████████████████████████████████| 596 kB 80.2 MB/s 
     |████████████████████████████████| 67 kB 6.0 MB/s 
     |████████████████████████████████| 895 kB 81.6 MB/s 
     |████████████████████████████████| 144 kB 94.6 MB/s 
     |████████████████████████████████| 181 kB 52.5 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 134 kB 79.4 MB/s 
     |████████████████████████████████| 243 kB 84.6 MB/s 
     |█████████████

In [3]:
from simpletransformers.ner.ner_utils import (
    InputExample,
    LazyNERDataset,
    convert_examples_to_features,
    get_examples_from_df,
    load_hf_dataset,
    read_examples_from_file,
    flatten_results,
)
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
import numpy as np
from tqdm.notebook import tqdm
import torch
from simpletransformers.ner import NERModel

In [4]:
class CustomNER(NERModel):
  def __init__(self,model_type,
        model_name,
        labels=None,
        weight=None,
        args=None,
        use_cuda=True,
        cuda_device=-1,
        onnx_execution_provider=None,
        **kwargs,):
    super().__init__(model_type,model_name,labels,weight,args,use_cuda,cuda_device,onnx_execution_provider,**kwargs)
  
  def predict(self, to_predict, split_on_space=True):
      """
      Performs predictions on a list of text.
      Args:
          to_predict: A python list of text (str) to be sent to the model for prediction.
          split_on_space: If True, each sequence will be split by spaces for assigning labels.
                          If False, to_predict must be a a list of lists, with the inner list being a
                          list of strings consisting of the split sequences. The outer list is the list of sequences to
                          predict on.
      Returns:
          preds: A Python list of lists with dicts containing each word mapped to its NER tag.
          model_outputs: A Python list of lists with dicts containing each word mapped to its list with raw model output.
      """  # noqa: ignore flake8"
 
      device = self.device
      model = self.model
      args = self.args
      pad_token_label_id = self.pad_token_label_id
      preds = None
 
      if split_on_space:
          if self.args.model_type == "layoutlm":
              predict_examples = [
                  InputExample(
                      i,
                      sentence.split(),
                      [self.args.labels_list[0] for word in sentence.split()],
                      x0,
                      y0,
                      x1,
                      y1,
                  )
                  for i, (sentence, x0, y0, x1, y1) in enumerate(to_predict)
              ]
              to_predict = [sentence for sentence, *_ in to_predict]
          else:
              predict_examples = [
                  InputExample(
                      i,
                      sentence.split(),
                      [self.args.labels_list[0] for word in sentence.split()],
                  )
                  for i, sentence in enumerate(to_predict)
              ]
      else:
          if self.args.model_type == "layoutlm":
              predict_examples = [
                  InputExample(
                      i,
                      sentence,
                      [self.args.labels_list[0] for word in sentence],
                      x0,
                      y0,
                      x1,
                      y1,
                  )
                  for i, (sentence, x0, y0, x1, y1) in enumerate(to_predict)
              ]
              to_predict = [sentence for sentence, *_ in to_predict]
          else:
              predict_examples = [
                  InputExample(
                      i, sentence, [self.args.labels_list[0] for word in sentence]
                  )
                  for i, sentence in enumerate(to_predict)
              ]
 
      if self.args.onnx:
 
          # Encode
          model_inputs = self.tokenizer.batch_encode_plus(
              to_predict,
              return_tensors="pt",
              padding=True,
              truncation=True,
              is_split_into_words=(not split_on_space),
          )
 
          # Change shape for batching
          encoded_model_inputs = []
          if self.args.model_type in ["bert", "xlnet", "albert", "layoutlm"]:
              for (input_ids, attention_mask, token_type_ids) in tqdm(
                  zip(
                      model_inputs["input_ids"],
                      model_inputs["attention_mask"],
                      model_inputs["token_type_ids"],
                  )
              ):
                  encoded_model_inputs.append(
                      (input_ids, attention_mask, token_type_ids)
                  )
          else:
              for (input_ids, attention_mask) in tqdm(
                  zip(model_inputs["input_ids"], model_inputs["attention_mask"])
              ):
                  encoded_model_inputs.append((input_ids, attention_mask))
 
          # Setup batches
          eval_sampler = SequentialSampler(encoded_model_inputs)
          eval_dataloader = DataLoader(
              encoded_model_inputs,
              sampler=eval_sampler,
              batch_size=args.eval_batch_size,
          )
          for batch in tqdm(
              eval_dataloader, disable=args.silent, desc="Running Prediction"
          ):
              if self.args.model_type in ["bert", "xlnet", "albert", "layoutlm"]:
                  inputs_onnx = {
                      "input_ids": batch[0].detach().cpu().numpy(),
                      "attention_mask": batch[1].detach().cpu().numpy(),
                      "token_type_ids": batch[2].detach().cpu().numpy(),
                  }
              else:
                  inputs_onnx = {
                      "input_ids": batch[0].detach().cpu().numpy(),
                      "attention_mask": batch[1].detach().cpu().numpy(),
                  }
 
              # Run the model (None = get all the outputs)
              output = self.model.run(None, inputs_onnx)
 
              if preds is None:
                  preds = output[0]
                  out_input_ids = inputs_onnx["input_ids"]
                  out_attention_mask = inputs_onnx["attention_mask"]
              else:
                  preds = np.append(preds, output[0], axis=0)
                  out_input_ids = np.append(
                      out_input_ids, inputs_onnx["input_ids"], axis=0
                  )
                  out_attention_mask = np.append(
                      out_attention_mask, inputs_onnx["attention_mask"], axis=0
                  )
          
          # out_label_ids = np.zeros_like(out_input_ids)
          # for index in range(len(out_label_ids)):
          #     out_label_ids[index][0] = -100
          #     out_label_ids[index][-1] = -100
 
          pad_token_label_id = -100
          out_label_ids = [[] for _ in range(len(to_predict))]
          max_len = len(out_input_ids[0])
 
          for index, sentence in enumerate(to_predict):
              for word in sentence.split():
                  word_tokens = self.tokenizer.tokenize(word)
                  out_label_ids[index].extend(
                  [0] + [pad_token_label_id] * (len(word_tokens) - 1)
              )
              out_label_ids[index].insert(0,pad_token_label_id)
              out_label_ids[index].append(pad_token_label_id)
 
              if len(out_label_ids[index]) < max_len:
                  out_label_ids[index].extend([-100] * (max_len-len(out_label_ids[index])))
 
          out_label_ids = np.array(out_label_ids).reshape(len(out_label_ids), max_len)
      else:
 
          eval_dataset = self.load_and_cache_examples(
              None, to_predict=predict_examples
          )
          eval_sampler = SequentialSampler(eval_dataset)
          eval_dataloader = DataLoader(
              eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size
          )
 
          self._move_model_to_device()
 
          eval_loss = 0.0
          nb_eval_steps = 0
          preds = None
          out_label_ids = None
          model.eval()
 
          if args.n_gpu > 1:
              model = torch.nn.DataParallel(model)
 
          if self.args.fp16:
              from torch.cuda import amp
 
          for batch in tqdm(
              eval_dataloader, disable=args.silent, desc="Running Prediction"
          ):
              batch = tuple(t.to(device) for t in batch)
 
              with torch.no_grad():
                  inputs = self._get_inputs_dict(batch)
 
                  if self.args.fp16:
                      with amp.autocast():
                          outputs = self._calculate_loss(
                              model,
                              inputs,
                              loss_fct=self.loss_fct,
                              num_labels=self.num_labels,
                              args=self.args,
                          )
                          tmp_eval_loss, logits = outputs[:2]
                  else:
                      outputs = self._calculate_loss(
                          model,
                          inputs,
                          loss_fct=self.loss_fct,
                          num_labels=self.num_labels,
                          args=self.args,
                      )
                      tmp_eval_loss, logits = outputs[:2]
 
                  if self.args.n_gpu > 1:
                      tmp_eval_loss = tmp_eval_loss.mean()
                  eval_loss += tmp_eval_loss.item()
 
              nb_eval_steps += 1
 
              if preds is None:
                  preds = logits.detach().cpu().numpy()
                  out_label_ids = inputs["labels"].detach().cpu().numpy()
                  out_input_ids = inputs["input_ids"].detach().cpu().numpy()
                  out_attention_mask = inputs["attention_mask"].detach().cpu().numpy()
              else:
                  preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
                  out_label_ids = np.append(
                      out_label_ids, inputs["labels"].detach().cpu().numpy(), axis=0
                  )
                  out_input_ids = np.append(
                      out_input_ids,
                      inputs["input_ids"].detach().cpu().numpy(),
                      axis=0,
                  )
                  out_attention_mask = np.append(
                      out_attention_mask,
                      inputs["attention_mask"].detach().cpu().numpy(),
                      axis=0,
                  )
 
          eval_loss = eval_loss / nb_eval_steps
      token_logits = preds
      preds = np.argmax(preds, axis=2)
 
      label_map = {i: label for i, label in enumerate(self.args.labels_list)}
 
      out_label_list = [[] for _ in range(out_label_ids.shape[0])]
      preds_list = [[] for _ in range(out_label_ids.shape[0])]
 
      for i in range(out_label_ids.shape[0]):
          for j in range(out_label_ids.shape[1]):
              if out_label_ids[i, j] != pad_token_label_id:
                  out_label_list[i].append(label_map[out_label_ids[i][j]])
                  preds_list[i].append(label_map[preds[i][j]])
 
      if split_on_space:
          preds = [
              [
                  {word: preds_list[i][j]}
                  for j, word in enumerate(sentence.split()[: len(preds_list[i])])
              ]
              for i, sentence in enumerate(to_predict)
          ]
      else:
          preds = [
              [
                  {word: preds_list[i][j]}
                  for j, word in enumerate(sentence[: len(preds_list[i])])
              ]
              for i, sentence in enumerate(to_predict)
          ]
 
      word_tokens = []
      for n, sentence in enumerate(to_predict):
          w_log = self._convert_tokens_to_word_logits(
              out_input_ids[n],
              out_label_ids[n],
              out_attention_mask[n],
              token_logits[n],
          )
          word_tokens.append(w_log)
 
      if split_on_space:
          model_outputs = [
              [
                  {word: word_tokens[i][j]}
                  for j, word in enumerate(sentence.split()[: len(preds_list[i])])
              ]
              for i, sentence in enumerate(to_predict)
          ]
      else:
          model_outputs = [
              [
                  {word: word_tokens[i][j]}
                  for j, word in enumerate(sentence[: len(preds_list[i])])
              ]
              for i, sentence in enumerate(to_predict)
          ]
 
      return preds, model_outputs

In [5]:
_POS_TAGS = ["NN", "VV", "PU", "CC", "PS", "AX", "AV", "FX", "NU", "AJ", "CL", "PR", "NG", "PA", "XX", "IJ"]

In [7]:
model_ONNX = CustomNER(
    "camembert",'/content/drive/MyDrive/POSTAG/BERT_model/ONNXModel(Noquantize)/', labels=_POS_TAGS)

/usr/local/lib/python3.7/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:56: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  "Available providers: '{}'".format(name, ", ".join(available_provider_names)))
/usr/local/lib/python3.7/dist-packages/simpletransformers/ner/ner_model.py:379: UserWarning: use_multiprocessing automatically disabled as CamemBERT fails when using multiprocessing for feature conversion.
  "use_multiprocessing automatically disabled as CamemBERT"


In [8]:
model = CustomNER(
    "camembert",'/content/drive/MyDrive/POSTAG/BERT_model/LST21BaseWholeFinetune/', labels=_POS_TAGS)

/usr/local/lib/python3.7/dist-packages/simpletransformers/ner/ner_model.py:379: UserWarning: use_multiprocessing automatically disabled as CamemBERT fails when using multiprocessing for feature conversion.
  "use_multiprocessing automatically disabled as CamemBERT"


In [9]:
text = ['ไ่ก่ จิก เด็ก ตาย คน _ เกิด บน ปาก โอ่ง']

In [10]:
result , _  = model.predict(text)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
result_ONNX , _  = model_ONNX.predict(text,split_on_space = True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


0it [00:00, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
print(result)

[[{'ไ่ก่': 'NN'}, {'จิก': 'VV'}, {'เด็ก': 'NN'}, {'ตาย': 'VV'}, {'คน': 'CL'}, {'_': 'PU'}, {'เกิด': 'VV'}, {'บน': 'PS'}, {'ปาก': 'NN'}, {'โอ่ง': 'NN'}]]


In [13]:
print(result_ONNX)

[[{'ไ่ก่': 'NN'}, {'จิก': 'VV'}, {'เด็ก': 'NN'}, {'ตาย': 'VV'}, {'คน': 'CL'}, {'_': 'PU'}, {'เกิด': 'VV'}, {'บน': 'PS'}, {'ปาก': 'NN'}, {'โอ่ง': 'NN'}]]
